# Install all dependencies

In [ ]:
!pip install --upgrade pip
!pip install torch==2.0.1
!pip install einops
!pip install flask
!pip install waitress
!pip install flask-restful

# Download Euterpe-X repo

In [ ]:
!git clone https://github.com/asigalov61/Euterpe-X

# Unzip Pre-Trained Euterpe X Model

In [ ]:
print('=' * 70)
%cd /home/ubuntu/Euterpe-X/Model

print('=' * 70)
print('Unzipping pre-trained Euterpe X model...Please wait...')

!cat /home/ubuntu/Euterpe-X/Model/Euterpe_X_Trained_Model.zip* > /home/ubuntu/Euterpe-X/Model/Euterpe_X_Trained_Model.zip
print('=' * 70)

!unzip -j /home/ubuntu/Euterpe-X/Model/Euterpe_X_Trained_Model.zip
print('=' * 70)

print('Done! Enjoy! :)')
print('=' * 70)
%cd /home/ubuntu/
print('=' * 70)

# API

In [ ]:
import torch

%cd /home/ubuntu/Euterpe-X
import TMIDIX
from x_transformer import TransformerWrapper, Decoder, AutoregressiveWrapper

%cd /home/ubuntu/

from flask import Flask, jsonify, request
from flask_restful import reqparse, abort, Api, Resource
from waitress import serve

In [ ]:
#@title Load Euterpe X Model
full_path_to_model_checkpoint = '/home/ubuntu/Euterpe-X/Model/Euterpe_X_Trained_Model_58000_steps_0.6865_loss.pth'

print('=' * 70)
print('Loading Euterpe X Pre-Trained Model...')
print('Please wait...')
print('=' * 70)
print('Instantiating model...')

SEQ_LEN = 2048

# instantiate the model

model = TransformerWrapper(
    num_tokens = 3344,
    max_seq_len = SEQ_LEN,
    attn_layers = Decoder(dim = 1024, depth = 32, heads = 8)
)

model = AutoregressiveWrapper(model)

model = torch.nn.DataParallel(model)

model.cuda()
print('=' * 70)
print('Done!')
print('=' * 70)

print('=' * 70)

print('Loading model checkpoint...')

model.load_state_dict(torch.load(full_path_to_model_checkpoint))
print('=' * 70)

model.eval()

print('Done!')
print('=' * 70)

In [ ]:
app = Flask(__name__)
api = Api(app)

# argument parsing

parser = reqparse.RequestParser()
parser.add_argument('input_notes', type=str, help='Note string as a list of ints. I.e: [3343, 3330, 3339]')
parser.add_argument('notes_count', type=int, help='Number of notes to generate')

class PredictSentiment(Resource):
    def post(self):

        json_data = request.get_json(force=True)
        input_notes = json_data['input_notes']
        notes_count = json_data['notes_count']
        
        print('=' * 70)
        print('Received API request to generate', notes_count, 'notes...')
        print('=' * 70)
        print('Request input notes sequence:')
        print('=' * 70)
        print(input_notes)
        print('=' * 70)
        
        # Model stuff
        
        num_batches = 1

        inp = [input_notes] * num_batches

        inp = torch.LongTensor(inp).cuda()

        out = model.module.generate(inp, 
                              notes_count*3, 
                              temperature=0.9, 
                              return_prime=False, 
                              verbose=True)

        out_notes = out[0].tolist() # 0 == batch number
        
        print('=' * 70)
        print('Done!')
        print('=' * 70)
        
        return jsonify(input_notes=input_notes, output_notes=out_notes)

# Setup the Api resource routing here
# Route the URL to the resource
api.add_resource(PredictSentiment, '/')

if __name__ == '__main__':
    print('Euterpe-X API Sever')
    serve(app, port=6000)